In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [107]:
results_df = pd.read_csv("Datasets/results.csv")

In [109]:
minimum_rank = 30

results_df = results_df[(results_df.rank_1<minimum_rank)&(results_df.rank_2<minimum_rank)]

results_df.reset_index(drop='True',inplace=True)

In [110]:
results_df.head()

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
0,2020-03-17,fnatic,BIG,Mirage,12,16,2,1,9,6,3,10,5226,2340397,5,18,1,2,2
1,2020-03-17,fnatic,BIG,Overpass,16,7,1,2,4,4,12,3,5226,2340397,5,18,1,2,2
2,2020-03-17,Natus Vincere,North,Nuke,10,16,2,1,7,8,3,8,5226,2340396,1,23,0,2,2
3,2020-03-17,Natus Vincere,North,Dust2,10,16,2,1,5,10,5,6,5226,2340396,1,23,0,2,2
4,2020-03-17,forZe,Complexity,Train,9,16,2,2,3,7,6,9,5226,2340279,24,17,2,1,1
